# Explore and Analyze Tornadoes from 2013 using MATLAB in Jupyter

## MATLAB in Jupyter

Welcome to your first Jupyter Notebook running MATLAB as the kernel.  This means that you can write all commands in the MATLAB language and MATLAB will do the computation for you.  More details on the MATLAB Kernel for Jupyter can be found in this blogpost: https://blogs.mathworks.com/matlab/2023/01/30/official-mathworks-matlab-kernel-for-jupyter-released/ 

The only difference between this Jupyter notebook and ones you may have used before is that ALL of the code shown here is MATLAB code.  Everything else is **exactly the same** including

* How you evaluate cells - Shift+Enter or CTRL+Enter
* Double clicking on text cells (such as this one) to edit Markdown

## Discussion of our problem

Today, we are going to work with historical data from storm events in the United States in 2013. The file includes over 40 different types of storm events, and contain information such as geographic location, date, damage cost in dollars, injuries, deaths, and text descriptions of the storm. 

There is a lot of missing data. For example, many events affect a large area, so no specific location is provided. As you explore the data, you'll see these represented as `NaN`, which stands for "not a number." Dealing with missing data is an important skill and there is a lot of functionality in MATLAB to help you do so.

This script explores the tornado events of 2013. There are many tornadoes in the data set. Nearly all tornado events have information for the location and damage associated with them. This makes tornadoes a great place to start. 

## Importing the data

### Today we'll use a Parquet file

The data is stored in the file `StormEvents.parquet` which is an example of a Parquet file. Parquet is in open-source column-oriented data storage format developed and maintained as a part of the Apache Software Foundation and is a great choice for 'Big Data' analysis. 

You may be more used to dealing with csv (comma separated values) files but the benefits of Parquet files are so great that the first thing I often do when given a .csv file is convert it to .parquet format! Of course, MATLAB has extensive support for CSV files if that's what you need but I wanted to start with a more modern approach today. 

Disk space alone is a good enough reason to prefer Parquet over csv, the file we will be using today would be 10x bigger if it was a csv file! There are, however many more benefits to using Parquet over .csv and you can read about some of them at https://blogs.mathworks.com/matlab/2023/05/05/working-efficiently-with-data-parquet-files-and-the-needle-in-a-haystack-problem/

In [ ]:
events = parquetread("/workspaces/codespaces-jupyter-matlab-workshop/data/StormEvents.parquet");

The `parquetread` file creates a table from the file called `events` which is of size 59,985 rows and 16 columns which we can see for ourselves by running the following commands

In [ ]:
class(events)

In [ ]:
size(events)

Displaying such large tables in the notebook can be problematic, particularly when each row takes up multiple lines, so let's just look at the first 7 columns.  Note that only the first few rows are displayed

In [ ]:
events(:,[1:7])

The list of column headings can be found as follows

In [ ]:
events.Properties.VariableNames

We can also request to view columns by their row name

In [ ]:
events(:,["Month","Begin_Date_Time","Property_Cost","Crop_Cost"])

Some ofthe data contains missing data, encoded as `NaN`.  Here are some that contain missing `Property_Cost` data.

In [ ]:
events(ismissing(events.Property_Cost),["Month","Begin_Date_Time","Property_Cost","Crop_Cost"])

## Data Processing steps

As you explore new data, you'll typically discover there is some "clean-up" to perform. Above, for example, we discovered that there was some missing data.  We'll do some clean up and a little more processing here

In [ ]:
% Put months in correct order
monthOrder = ["January", "February", "March", "April", "May", "June", "July",...
    "August", "September", "October", "November", "December"];
events.Month = reordercats(events.Month, monthOrder);

% Set missing Property and Crop Cost to $0
events.Property_Cost(ismissing(events.Property_Cost)) = 0;
events.Crop_Cost(ismissing(events.Crop_Cost)) = 0;

% Add total damage to the table
events.Total_Damage = events.Property_Cost + events.Crop_Cost;

Description of code above

* By default, the text in the "Month" variable of the table is ordered alphabetically. The code reorders the categories to match the calendar. This is not necessary but makes the visualizations easier to interpret. 
* Some events are missing information on the associated property and/or damage caused to property and crops. Missing cost is replaced with 0 as missing information likely means small or no damage to report.
* A new variable named Total_Damage is added to the table. This variable is the sum of Property_Cost + Crop_Cost.

# Visualize the Locations of Tornadoes
## Plot all tornados above a damage threshold

The code below creates a plot of tornado locations on a map. The size of the markers represents the cost in dollars of the damage caused by the tornado. 

The `minDamage` variable represents minimum value for total damage of an event to be included in the plot. 

In [ ]:
minDamage = 0;    % modify this to change which tornados are included in the plot

% Select tornadoes above a damage threshold set by the minDamage variable
tornadoes = events(events.Event_Type == "Tornado" & events.Total_Damage >= minDamage, :);
% Plot the results on a map
geobubble(tornadoes.Begin_Lat,tornadoes.Begin_Lon,tornadoes.Total_Damage, tornadoes.Month);
title("Tornadoes with cost >= $" + minDamage)

## Plot tornadoes for a given month
    
Change the `eventMonth` variable and rerun the cell to observe how the number and location of tornadoes varies throughout the year. 
All tornadoes are treated as equal regardless of damage caused.

In [ ]:
eventMonth = "June";   % Change this and re-run the cell

tornadoes = events(events.Event_Type == "Tornado", :);
tornadoes = tornadoes(tornadoes.Month == eventMonth, :);
geobubble(tornadoes.Begin_Lat,tornadoes.Begin_Lon);
title("Tornadoes in the month of " + eventMonth)

# Calculate Group Summary Statistics

## Create a histogram of tornado events

In the section above, you may have noticed that some months have more tornadoes than others. This section creates a histogram to show the number of tornadoes by month.
* Run this section. Which month has the most tornadoes? Is there a tornado season?

In [ ]:
tornadoes = events(events.Event_Type == "Tornado",:);
    
%the tornadoes table has a huge number of columns which can be difficult to displaye, so we will just choose a few columns
tornadoes(:,["Month","Event_Type","Begin_Date_Time","Timezone"])

In [ ]:
histogram(tornadoes.Month);
title("Tornados by Month")
xlabel("Month")
ylabel("Number of Tornadoes")

# Total damage by month

Looking at the histogram, the month of May had far more tornadoes than any other month. Now examine the damage caused by these tornadoes. 

Change the statistic and re-evaluate the cell

In [ ]:
stat = "Max"   % Could also be "Sum", "Mean" or "Max"
tornadoes = events(events.Event_Type == "Tornado",:);
% Group tornadoes by month and calculate the selected stat
cost = groupsummary(tornadoes, "Month", stat, "Total_Damage")

% View the results using a bar chart
bar(cost.Month, cost{:, end})
title(stat + " of tornado cost by month")
ylabel("Cost in dollars")

# Closing Remarks

Notice how quickly creating a variety of visualizations provided an overview of lots of data. What have you learned about tornadoes in the United States in 2013? Hopefully you gained a feel for the data. This is important later when trying to evaluate predictive models. Without a feel for the data, it's difficult to look at the output of a complex model and determine if the results are reasonable.

For now, you can state that in 2013:

* The frequency of tornadoes was less during the winter months than late spring/early summer
* Winter tornadoes tend to be in the southern United States
* An extremely damaging tornado occurred in November

Maybe you have a lot more questions. For example, what states are most affected? What do the results look like for different years? This script just looks at tornadoes, what about all the other storm events? The fun, and challenging, aspect of having lots of data is trying to explore all the possible questions and relationships.

## Want to learn more? Try our Coursera course. 

If you enjoyed this example and would like to learn more about data analysis in MATLAB, maybe try our Coursera course at https://www.coursera.org/learn/exploratory-data-analysis-matlab